In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
import prometheux_chain as px

PMTX_TOKEN = "dev-token"
os.environ["PMTX_TOKEN"] = PMTX_TOKEN

#### Project functions

In [2]:
px.cleanup_projects()

All user's projects deleted successfully


In [3]:
project_id = px.save_project(project_name="test_project")
project_id

'1e0e4e01aa8'

In [4]:
px.list_projects()

[{'id': '1e0e4e01aa8',
  'name': 'test_project',
  'author': 'devuser',
  'scope': 'user',
  'to_persist': True,
  'created_at': '2025-11-18T18:35:50.620954+00:00',
  'updated_at': '2025-11-18T18:35:50.620954+00:00'}]

In [5]:
project = px.load_project(project_id=project_id)
project

{'id': '1e0e4e01aa8',
 'name': 'test_project',
 'author': 'devuser',
 'scope': 'user',
 'to_persist': True,
 'created_at': '2025-11-18T18:35:50.620954+00:00',
 'updated_at': '2025-11-18T18:35:50.620954+00:00'}

#### Config functions

In [6]:
px.load_user_config()

{}

#### Data functions

In [7]:
px.cleanup_sources()

All data sources deleted successfully


In [8]:
db = px.data.database.Database(
    database_type="csv",
    username="",
    password="",
    host=os.getcwd() + "/../resources/ownership",
    port=0000,
    database_name="ownership.csv",
    options={} #{"delimiter": "\t"}
)
px.connect_sources(database_payload=db)

{'connectionStatus': True,
 'sources': [{'id': '8daab3d610af52f0b3c2b30710311724',
   'bind_annotation': '@bind("ownership_csv","csv useHeaders=\'true\'","/Users/teodorobaldazzi/Documents/workdir/prometheux/prometheux_chain/test/notebooks/../resources/ownership","ownership.csv").',
   'model_annotation': '',
   'rule_block': '',
   'row_count': 0,
   'column_count': 3,
   'author': 'devuser',
   'timestamp': '2025-11-18T18:35:51.046454+00:00'}],
 'errorMessage': None}

In [9]:
px.list_sources()

[{'id': '8daab3d610af52f0b3c2b30710311724',
  'bind_annotation': '@bind("ownership_csv","csv useHeaders=\'true\'","/Users/teodorobaldazzi/Documents/workdir/prometheux/prometheux_chain/test/notebooks/../resources/ownership","ownership.csv").',
  'model_annotation': '',
  'rule_block': '',
  'row_count': 0,
  'column_count': 3,
  'author': 'devuser',
  'timestamp': '2025-11-18T18:35:51.046454+00:00'}]

In [10]:
px.infer_schema(database=db, add_bind=True, add_model=False)

'%%\n@bind("ownership_csv","csv useHeaders=\'true\'","/Users/teodorobaldazzi/Documents/workdir/prometheux/prometheux_chain/test/notebooks/../resources/ownership","ownership.csv").\nownership_csv_head(Owner,Company,Share) :- ownership_csv(Owner,Company,Share).\n@output("ownership_csv_head").\n@post("ownership_csv_head","limit(10)").\n%%\n\n'

#### Concept functions

In [11]:
px.cleanup_concepts(project_id=project_id)

In [12]:
concept_logic = """
@param("test","John").
ownership(Owner,Company,Shares) :- ownership_csv(Owner,Company,Shares), contains(Owner, ${test}).
@output("ownership").
"""

px.save_concept(project_id=project_id, concept_logic=concept_logic)

concept_logic = """
owner(Owner) :- ownership(Owner,Company,Shares).
@output("owner").
"""

px.save_concept(project_id=project_id, concept_logic=concept_logic)

concept_logic = """
company("Apple", "Redwood City, CA").
company("Google", "Mountain View, CA").
company("Microsoft", "Redmond, WA").
company("Amazon", "Seattle, WA").
company("Facebook", "Menlo Park, CA").
company("Twitter", "San Francisco, CA").
company("LinkedIn", "Sunnyvale, CA").
company("Instagram", "Menlo Park, CA").

location(Location) :- company(_,Location).

@output("location").
"""

px.save_concept(project_id=project_id, concept_logic=concept_logic)

In [13]:
px.run_concept(project_id=project_id, concept_name="owner", persist_outputs=False)

{'message': "Successfully ran concept 'owner'",
 'concept_name': 'owner',
 'predicates_populated': ['owner'],
 'total_records': 2,
 'evaluation_results': {'id': 77,
  'resultSet': {'owner': [['Alice Johnson'], ['John Doe']]},
  'types': {'owner': ['STRING']},
  'columnNames': {'owner': ['Owner']}}}

In [14]:
px.run_concept(project_id=project_id, concept_name="location", persist_outputs=True)

{'message': "Successfully ran concept 'location'",
 'concept_name': 'location',
 'predicates_populated': [],
 'total_records': 0,
 'evaluation_results': {'id': 78,
  'resultSet': {},
  'types': {'location': ['STRING']},
  'columnNames': {'location': ['Location']}}}

In [15]:
concept_logic = """
edge("a","b").
edge("b","c").
edge("c","d").
edge("d","e").
edge("a","d").
edge("f","g").
edge("g","h").
edge("h","i").

node("a").
node("b").
node("c").
node("d").
node("e").
node("f").
node("g").
node("h").
node("i").

edge_undirected(X,Y) :- edge(X,Y).
edge_undirected(Y,X) :- edge(X,Y).
cc(X, Z) :- node(X).
Z1 = Z2  :- cc(X, Z1), edge_undirected(X, Y), cc(Y, Z2).

@output("cc").
"""

px.save_concept(project_id=project_id, concept_logic=concept_logic)
px.run_concept(project_id=project_id, concept_name="cc", persist_outputs=False)

{'message': "Successfully ran concept 'cc'",
 'concept_name': 'cc',
 'predicates_populated': ['cc'],
 'total_records': 9,
 'evaluation_results': {'id': 79,
  'resultSet': {'cc': [['i', 'z_1238543893_1'],
    ['h', 'z_1238543893_1'],
    ['d', 'z_1910318733_1'],
    ['e', 'z_1910318733_1'],
    ['a', 'z_1910318733_1'],
    ['g', 'z_1238543893_1'],
    ['f', 'z_1238543893_1'],
    ['c', 'z_1910318733_1'],
    ['b', 'z_1910318733_1']]},
  'types': {'cc': ['none', 'none']},
  'columnNames': {'cc': ['X', 'Z']}}}

In [16]:
python_script = """
import json

def main():
    # Simple calculations
    numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    
    # Calculate basic statistics
    total = sum(numbers)
    average = total / len(numbers)
    maximum = max(numbers)
    minimum = min(numbers)
    
    # Create results as a list of dictionaries (tabular format)
    results = [
        [
            str(numbers),
            total,
            round(average, 2),
            maximum,
            minimum,
            len(numbers)
        ]
    ]
    
    return results
"""

px.save_concept(project_id=project_id, 
               concept_logic="@output(\"test_python\").\ntest_python(Numbers,Total,Average,Maximum,Minimum,Count) :- python:run(\"python_script\").",
               python_scripts={
                    "python_script": python_script
                })

px.run_concept(project_id=project_id, 
                concept_name="test_python", 
                persist_outputs=False)

{'message': "Python concept 'test_python' executed successfully",
 'concept_name': 'test_python',
 'predicates_populated': ['test_python'],
 'total_records': 1,
 'python_execution': True,
 'evaluation_results': {'resultSet': {'test_python': [{'Numbers': '[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]',
     'Total': '55',
     'Average': '5.5',
     'Maximum': '10',
     'Minimum': '1',
     'Count': '10'}]},
  'columnNames': {'test_python': ['value']}}}

In [17]:
px.list_concepts(project_id=project_id)

[{'predicate_name': 'ownership',
  'fields': {'Owner': 'unknown', 'Company': 'unknown', 'Shares': 'unknown'},
  'column_count': 3,
  'is_input': False,
  'row_count': 0,
  'type': 'api',
  'timestamp': '2025-11-18T18:35:55.069417+00:00',
  'author': 'devuser',
  'description': 'The #ownership concept represents the ownership details of individuals or entities, including the owner, the company they own shares in, and the number of shares, filtered to include only those owners whose names contain the specified parameter value (${test}).'},
 {'predicate_name': 'owner',
  'fields': {'Owner': 'unknown'},
  'column_count': 1,
  'is_input': False,
  'row_count': 0,
  'type': 'api',
  'timestamp': '2025-11-18T18:35:56.826909+00:00',
  'author': 'devuser',
  'description': 'Represents individuals or entities identified as owners, derived from the #ownership concept by extracting the Owner attribute.'},
 {'predicate_name': 'location',
  'fields': {'Location': 'string'},
  'column_count': 1,
  'i

##### KG functions

In [18]:
px.save_kg(project_id=project_id, concepts=["owner", "location"])

{'status': 'success',
 'message': 'Virtual knowledge graph saved successfully',
 'data': None}

In [19]:
px.load_kg(project_id=project_id)

{'status': 'success',
 'message': 'Knowledge graph loaded successfully',
 'data': {'project_id': '1e0e4e01aa8',
  'concepts': ['owner', 'location'],
  'timestamp': '2025-11-18T18:36:05.443483+00:00',
  'author': 'devuser'}}

#### Copy, export, import projects functions

In [20]:
px.copy_project(project_id=project_id, new_project_name="copied_project")

{'new_project_id': '1ff31e22dc4'}

In [21]:
exported_workspace = px.export_workspace()
exported_workspace

{'scope': 'user',
 'extraction_timestamp': '2025-11-18T18:36:05.571279+00:00',
 'database_info': {'type': 'postgresql',
  'host': 'localhost',
  'port': '5432',
  'database': 'devuser'},
 'workspace_level_tables': ['projects_workspace_id'],
 'projects': {'1e0e4e01aa8': {'project_id': '1e0e4e01aa8',
   'scope': 'user',
   'extraction_timestamp': '2025-11-18T18:36:05.602962+00:00',
   'database_info': {'type': 'postgresql',
    'host': 'localhost',
    'port': '5432',
    'database': 'devuser'},
   'tables': {'datasources_workspace_id': {'schema': [{'name': 'datasource_id',
       'type': 'text',
       'nullable': 'NO',
       'default': None},
      {'name': 'datasource_type',
       'type': 'text',
       'nullable': 'YES',
       'default': None},
      {'name': 'host', 'type': 'text', 'nullable': 'YES', 'default': None},
      {'name': 'port', 'type': 'text', 'nullable': 'YES', 'default': None},
      {'name': 'username', 'type': 'text', 'nullable': 'YES', 'default': None},
      {'

In [22]:
px.import_workspace(exported_workspace)

{'status': 'success',
 'message': 'Workspace tables imported successfully. Imported 2 projects (0 failed), created 0 tables, inserted 0 rows.',
 'data': {'target_scope': 'user',
  'injection_timestamp': '2025-11-18T18:36:05.669358+00:00',
  'tables_processed': {'projects_workspace_id': {'status': 'merged',
    'target_table_name': 'projects_workspace_id',
    'rows_inserted': 0,
    'rows_updated': 2}},
  'projects_imported': {'1e0e4e01aa8': {'status': 'success',
    'new_project_id': '1e0e4e01aa8',
    'summary': {'tables_created': 0,
     'tables_merged': 4,
     'tables_failed': 0,
     'rows_inserted': 0,
     'rows_updated': 12}},
   '1ff31e22dc4': {'status': 'success',
    'new_project_id': '1ff31e22dc4',
    'summary': {'tables_created': 0,
     'tables_merged': 4,
     'tables_failed': 0,
     'rows_inserted': 0,
     'rows_updated': 12}}},
  'summary': {'tables_created': 0,
   'tables_merged': 9,
   'tables_failed': 0,
   'rows_inserted': 0,
   'rows_updated': 26,
   'projects

#### GraphRAG function

In [23]:
graph_rag_result = px.graph_rag(project_id=project_id,
                                  question="John Doe", 
                                  rag_concepts=[{"concept": "ownership", "field_to_embed": "Owner"}],
                                  graph_selected_concepts=["owner"],
                                  top_k=5)

print(graph_rag_result['graph_rag_results'])

{'owner(Owner)': [['Alice Johnson'], ['John Doe']]}
